In [82]:
!pip install timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from PIL import Image
import os
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification
import timm

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

import timm

model = timm.create_model("hf_hub:timm/maxvit_tiny_tf_224.in1k", pretrained=True)

model.to(device)  # Move model to GPU

# Define optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Define loss function
criterion = nn.CrossEntropyLoss()

Using device: cuda


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [83]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SSA(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SSA, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0)

    def forward(self, inputs):
        batch_size, channels, height, width = inputs.size()
        q = k = v = self.conv(inputs.permute(0, 3, 1, 2))
        Qshape = q.size()
        Kshape = k.size()
        Vshape = v.size()
        a = Qshape[2] * Qshape[3]
        q = q.view(batch_size, Qshape[1], a)
        k = k.view(batch_size, Kshape[1], a).permute(0, 2, 1)
        qk = torch.matmul(q, k)
        qk = F.softmax(qk, dim=-1)
        v = v.view(batch_size, Vshape[1], a)
        qkv = torch.matmul(qk, v)
        qkv = qkv.view(batch_size, Vshape[1], Vshape[2], Vshape[3])
        qkv = qkv.permute(0, 1, 2, 3)
        qkv = self.conv(qkv)
        qkv = qkv.permute(0,3,2,1)
        return qkv

# Example usage:
inputs = torch.randn(1, 14, 14, 128)  # Assuming batch size of 1
ssa = SSA(in_channels=128, out_channels=128)
print("Input Shape", inputs.shape)
output = ssa(inputs)
print("Output shape:", output.shape)

Input Shape torch.Size([1, 14, 14, 128])
Output shape: torch.Size([1, 14, 14, 128])


In [84]:
class CDSA(nn.Module):
    def __init__(self, fltr, nh):
        super(CDSA, self).__init__()
        self.fltr = fltr
        self.nh = nh
        self.attn = nn.ModuleList([SSA(fltr // nh, fltr // nh) for _ in range(nh)])  # Self-Attention modules
        self.conv = nn.Conv2d(fltr, fltr, kernel_size=1, stride=1, padding=0)
        self.relu = nn.ReLU()

    def forward(self, input_tensor):
        attn = []
        feature_split = torch.chunk(input_tensor, self.nh, dim=3)  # Split input tensor along channel dimension
        shape = feature_split[0].shape

        # Apply self-attention to each split of the input tensor
        x = self.attn[0](feature_split[0])
        attn.append(x)

        for i in range(1, self.nh):
            x = feature_split[i] + x  # Residual connection
            x = self.attn[i](x)  # Apply self-attention
            attn.append(x)

        # Concatenate the outputs of self-attention along the channel dimension
        mh_lka_attn = torch.cat(attn, dim=3)

        # Apply 1x1 convolution followed by ReLU activation
        mh_lka_attn = mh_lka_attn.permute(0,3,2,1)
        mh_lka_attn = self.conv(mh_lka_attn)
        mh_lka_attn = self.relu(mh_lka_attn)
        mh_lka_attn = mh_lka_attn.permute(0,3,2,1)
        return mh_lka_attn

In [85]:
import torch
import torch.nn as nn
import timm
from torchsummary import summary

# Define the CAL class
class CAL(nn.Module):
    def __init__(self):
        super(CAL, self).__init__()

    def forward(self, x):
        fltr = 256
        nh = 2
        rs1 = x = torch.add(x, x)
        layer_norm1 = nn.LayerNorm(x.size()[1:], eps=1e-6).to(x.device)  # Move to the same device as x
        x = layer_norm1(x)

        # Assuming CDSA is defined elsewhere
        cdsa = CDSA(fltr, nh).to(x.device)  # Move to the same device as x
        cdsa_output = cdsa(x)
        x = cdsa_output

        rs2 = x = rs1 + x
        layer_norm2 = nn.LayerNorm(x.size()[1:], eps=1e-6).to(x.device)  # Move to the same device as x
        x = layer_norm2(x)

        x = x.permute(0, 3, 2, 1)
        conv = nn.Conv2d(fltr, fltr, kernel_size=1, padding=0).to(x.device)  # Move to the same device as x
        x = conv(x)
        x = x.permute(0, 3, 2, 1)

        output = rs2 + x
        return output

In [86]:
loss_fun= 'categorical_crossentropy'
gpu_num=1
k=5
lr1=0.005
lr2=0.0001
image_size=224
classes=8
ratio=8
fltr=256
nh=2  # number of splits
mag='40'

In [87]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define the MyModel class
class MyModel(nn.Module):
    def __init__(self, num_classes, fltr=256, nh=2):
        super(MyModel, self).__init__()
        self.model1 = timm.create_model("hf_hub:timm/maxvit_tiny_tf_224.in1k", pretrained=True).to(device)

        self.activation = {}
        def get_activation(name):
            def hook(module, input, output):
                self.activation[name] = output.detach()
            return hook

        layer_name = 'stages.2.blocks.4.attn_grid.drop_path2'
        desired_layer = self.model1.stages[2].blocks[4].attn_grid.drop_path2
        desired_layer.register_forward_hook(get_activation(layer_name))

        self.conv = nn.Conv2d(256, fltr, 1).to(device)
        self.CAL_out = CAL().to(device)
        self.pool = nn.AdaptiveAvgPool2d((1,1)).to(device)
        self.fc = nn.Linear(fltr, num_classes).to(device)

    def forward(self, x):
        output = self.model1(x)
        activation_output = self.activation['stages.2.blocks.4.attn_grid.drop_path2']
        print(activation_output.shape)
        activation_output = activation_output.permute(0, 3, 2, 1)
        mn_output = self.conv(activation_output)
        mn_output = mn_output.permute(0, 3, 2, 1)
        print(mn_output.shape)
        mn_output = self.CAL_out(mn_output)
        print(mn_output.shape)
        mn_output= mn_output.permute(0, 3, 2, 1)
        # Apply adaptive average pooling
        adaptive_pool = nn.AdaptiveAvgPool2d(1)
        mn_output = adaptive_pool(mn_output)
        print(mn_output.shape)
        output = mn_output.view(m.size(0), -1)
        print("final",output.shape)
        return output

# Instantiate MyModel
classes = 8
model = MyModel(num_classes=classes)

# Move the model1 to the appropriate device
model.to(device)

# Print model1 summary
summary(model, input_size=(3, 224, 224))

Using device: cuda
torch.Size([2, 14, 14, 256])
torch.Size([2, 14, 14, 256])
torch.Size([2, 14, 14, 256])
torch.Size([2, 256, 1, 1])
final torch.Size([2, 256])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
        Conv2dSame-1         [-1, 64, 112, 112]           1,792
          Identity-2         [-1, 64, 112, 112]               0
          GELUTanh-3         [-1, 64, 112, 112]               0
    BatchNormAct2d-4         [-1, 64, 112, 112]             128
            Conv2d-5         [-1, 64, 112, 112]          36,928
              Stem-6         [-1, 64, 112, 112]               0
     AvgPool2dSame-7           [-1, 64, 56, 56]               0
          Identity-8           [-1, 64, 56, 56]               0
      Downsample2d-9           [-1, 64, 56, 56]               0
         Identity-10         [-1, 64, 112, 112]               0
         Identity-11         [-1, 64, 112, 112]               0
   Batc